In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sts
from matplotlib import rcParams,font_manager
from matplotlib.ticker import FormatStrFormatter
import pickle
import pandas as pd

%load_ext autoreload
%autoreload 2

In [3]:
rcParams['figure.figsize'] = (6,6) #figure size in inches
rcParams['font.family'] = "sans-serif" 
rcParams['font.weight'] = "roman" 
rcParams['font.style'] = "normal" # not 100% sure what this does but it was recommended
rcParams['font.size'] = 32 # not 100% sure what this does but it was recommended
rcParams['pdf.fonttype'] = 42
rcParams['axes.linewidth'] = 2 #thickness of the border
rcParams['xtick.major.width'] = rcParams['axes.linewidth'] # make sure line widths are consistent
rcParams['ytick.major.width'] = rcParams['axes.linewidth']
rcParams['axes.spines.right'] = False #hides right border
rcParams['axes.spines.top'] = False #hides top
rcParams['legend.frameon'] = False #hides box around the legend
rcParams['legend.fontsize'] = 18 #font size in pt
rcParams['axes.labelsize'] = 32 
rcParams['xtick.labelsize'] = 24
rcParams['ytick.labelsize'] = rcParams['xtick.labelsize']
rcParams['lines.linewidth'] = 3
rcParams['xtick.major.size'] = 5
rcParams['lines.markersize'] = 16
rcParams['ytick.major.size'] = rcParams['xtick.major.size'] 

In [6]:
direc = 'C:/Users/svo/Google Drive/Bas Games/'


In [71]:
with open(direc + 'Data/data_eye.cpp') as f:
    data = f.read()
data=data[data.find('{{{'):]
data=data[:data.find('}}'):]
data = data.replace('{','').replace(',\n','\n').replace('}','').replace('0x','').replace('ULL','').replace('BLACK','Black').replace('WHITE','White').split()
data = [line.split(',') for line in data]
df = pd.DataFrame(data,columns=['bp','wp','move','color','rt','id','group'])
df = df[['bp','wp','color','move','group','rt','id']]
df['bp']=df['bp'].apply(lambda x: int(x,16))
df['wp']=df['wp'].apply(lambda x: int(x,16))
df['move']=df['move'].apply(lambda x: int(x,16))

In [73]:
df.to_csv(direc + 'Data/data_eye.txt',header=None,index=False,sep='\t',line_terminator='\n')